# Using the shell in a notebook
We are going to be doing everything in a shell today - running existing programs.  There are at several ways to access the shell from within a notebook.

Unfortunately we do not have a shell notebook! We can run shell commands in a Python notebook if we explicitly tell it when we want to run commands on the shell (otherwise it will assume that the command is  Python).

The best way to tell the notebook that we want to run commands at the shell is to start a cell with `%%bash`, then jupyter knows to run the rest of the commands in the shell (bash is the name of the shell we are using).



In [ ]:
%%bash
echo "Hello World"

Note that if we try to do this without the `%%bash`, things get ugly

In [ ]:
echo "Hello World"

## Special Shell Commands
Some shell commands will work directly in jupyter, but this is just a preset collection of commands, available for our convenience.  A few of these are:
* `pwd` to print out
* `ls` to list the files in our current directory

In [ ]:
pwd

In [ ]:
ls

#A new directory

Now let's get started on the bioinformatics analysis!

First let's make a directory for us to work in so that we keep everything from this session in one place, and don't clutter our home directory with the files we are going to generate.


In [ ]:
%%bash
mkdir -p ~/bioinf_1

`mkdir` makes a new directory. 

`-p` tells mkdir to not give an error if the directory we tell it to make already exists.

`~/bioinf_1` tells mkdir that we want to make a new directory named "bioinf_1" in our home directory ("~" is a nickname for our home directory)

Now let's check to be sure that worked.  We will run `ls` and check that there is a "bioinf_1" directory now.

In [ ]:
ls

The next thing that we need to do is move into our new directory.  To do this, we use the `cd` command (change directory).  In theory we could do this in a `%%bash` cell, but it is imporant (for somewhat complicated reasons) that we do it in a regular cell so that subsequent commands will also run in our new directory.

In [ ]:
cd ~/bioinf_1/

And let's run `pwd` to be sure we have ended up in the right place

In [ ]:
%%bash
pwd

# Quality Control
For our first pass at analysis, we are just going to be working with the first read data (R1) from one sample.

To make things convenient, we are going to make a link in our current directory that points to the data file.  If you have ever used a file alias on a Mac or a file shortcut on Windows, this is the same thing!

In [ ]:
%%bash
ln -s ~/test_run_data/demux_2mismatch/r1.GTGAAA.fastq.gz r1.test.fastq.gz
ls

Let's take a quick look at our data.  This is something that is a little harder to do within the notebook than is necessary, so we are going to briefly switch to a terminal.  

To switch to a terminal, click on the jupyter "File" menu, and select "Open".  A new browser window/tab should open, with your jupyter "home base". Here, you should click on the "Files" tab if it is not already active, there click on "New" and select "Terminal", which should open a new live terminal.

### Tab completion (a brief aside)
If you push the tab key, the shell will fill in the file name that fits the prefix you typed, this can save you a lot of typing! For example, if you type `cd ~/bioi`, the the tab-key, the shell should automagically extend what you typed to `cd ~/bioinf_1`


## Back to our FASTQ file . . .
At the terminal's command prompt, type `cd ~/bioinf_1`.  Then type `zless r1.test.fastq.gz`.  You should see the first few lines of the file, notice that it looks like the examples we saw in lecture.

`zless` (and its standard cousin `less`, see below) can do a lot of things.  Here are a few important keystrokes:

* q : quit
* space : scroll down a page
* up/down arrow : scroll up/down by a line 

### Compression: gzip, zless, etc (another brief aside)
The ".gz" at the end of the FASTQ file name indicates that the fastq file was compressed using a program named gzip.  This is pretty common because FASTQ files can be huge.  `less` is a program for viewing text files, `zless` is a special version of this program that lets you view compressed text files without first decompressing them.

## Fastqc
Now that we have had a quick look at our fastq file, let's run some basic analysis using a program called fastqc.  This will take less than a minute to run.  While it is running it will say "In [\*]:" in the left margin, once it is done, a number will replace the asterisk.

In [ ]:
%%bash
fastqc --threads 2 --quiet r1.test.fastq.gz

Now, click here to view the [fastqc report](/files/bioinf_1/r1.test_fastqc/fastqc_report.html).

# Trimming and Filtering
Now we get into some actual preprocessing.  We will use `fastq-mcf` to trim adapter from our reads and do some quality filtering.  We need to trim adapter, because if a fragment is short enough, we will sequence all the way through the fragment and into the adapter.  Obviously the adapter sequence in not found in the genome, and can keep the read from aligning properly.  To do the trimming, we need to generate an adapter file.

## Making an adapter file
The first step is to get the adapter sequence.  For our test file, the index is 19 (barcode GTGAAA), this is from NEBNext Multiplex Oligos for Illumina (Index Primers Set 2).  So let's look in the manual: 

https://www.neb.com/~/media/Catalog/All-Products/6B6FC6C03B274E7FA0FDBF13015AB194/Datacards%20or%20Manuals/manualE7500.pdf

We will get the sequence for the universal primer and the index primer.  Now we need to make the adapter file; this needs to be in FASTA format.

1. click on the jupyter "File" menu, and select "Open".  
2. When the the new browser window/tab opens, click on the "Files" tab if it is not already active.
3. Click on "bioinf_1", then in the "New" menu select "Text File".
4. In this text file, paste the primer name and sequence.  
5. Now clean up by putting a ">" before the name, put the sequence on the next line, remove spaces and non-sequence characters.
6. Click on "untitled.txt" to change the file name to "neb_19_adapter.fasta"
7. Save the file.

When we work with more samples, we will need to include the adapter sequence for each one

## Running fastq-mcf
You can run `fastq-mcf -h` to get details about running fastq-mcf.  So how are we running fastq-mcf?  We will adjust run parameters, because some of the defaults set a low bar (even the author acknowleges this).

1. neb_19_adapter.fasta : the adapter file
2. r1.test.fastq.gz : the FASTQ with the data (fastq-mcf, like most NGS analysis software, detects gzipped files and automatically decompresses on the fly)
3. -q 20 : if a read has any bases with quality score lower than this, trim them and anything 3' 
4. -x 0.5 : if this percentage (or higher) of the reads have an "N" in a given position, trim all reads to that position
5. -o r1.test.trim.fastq.gz : output file (the .gz ending tells fastq-mcf to compress the output file)

In [ ]:
%%bash
fastq-mcf neb_19_adapter.fasta r1.test.fastq.gz -q 20 -x 0.5 -o r1.test.trim.fastq.gz

at this point we could run fastqc on the output of fastq-mcf to see if statistics have improved, but we will skip that for now.

# What do quality scores mean?


In [ ]:
def error_prob(quality):
    qval = ord(quality)-33
    return 10**(qval/-10.0)

for phred in range(0,42):
    qual = chr(phred+33)
    print "{0}\t{1}\t{2}".format(qual, phred, error_prob(qual))